<a href="https://colab.research.google.com/github/mjag7682/CS9-1-NLP-for-Twitter-Data-for-predicting-stocks/blob/Approach2_Test/Labelling_Approach2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import requests 
from pprint import pprint
import pandas as pd
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
#https://drive.google.com/file/d/1vPGsUkX5jUjSV7PqenHy1GpQvM0CfLit/view?usp=sharing
id = '1vPGsUkX5jUjSV7PqenHy1GpQvM0CfLit'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('stocktwits_PG.csv')
df_stocktwitsPG = pd.read_csv('stocktwits_PG.csv')
# https://drive.google.com/file/d/18BfEHPgpjfJRNIiH1ZsFL0CiKqpLwzDf/view?usp=sharing
id = '18BfEHPgpjfJRNIiH1ZsFL0CiKqpLwzDf'
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('PG_1.csv')
df_PG = pd.read_csv('PG_1.csv')
df_PG.head()
PG_st_Date = df_stocktwitsPG['datetime'].tolist()
PG_message = df_stocktwitsPG['message'].tolist()
PG_Open = df_PG['Open'].tolist()
PG_Close = df_PG['Close'].tolist()
PG_Date = df_PG['Date'].tolist()


In [ ]:
df_PG.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2009-08-10,51.919998,52.279999,51.580002,52.169998,37.088398,11763800
1,2009-08-11,52.110001,52.279999,51.930000,51.930000,36.917786,9293400
2,2009-08-12,52.000000,52.490002,51.849998,52.080002,37.024426,10148400
3,2009-08-13,52.290001,52.340000,51.750000,52.299999,37.180828,10486500
4,2009-08-14,52.549999,52.810001,51.869999,52.369999,37.230595,12069600


In [ ]:
df_stocktwitsPG.head()

,symbol,message,datetime,user,message_id
0,PG,TOP DOW 30 Stocks with the highest Payout Rati...,2020-07-21T21:37:30Z,1550174,229729267
1,PG,$PG Green to red move: 1.11 percentage to -...,2020-07-21T20:28:32Z,1442893,229707089
2,PG,"$PG Bought. Hopefully, Ol&#39; Gil made the r...",2020-07-21T19:49:53Z,2926317,229689416
3,PG,$PG 322 overnight open 321 then down 319,2020-07-21T19:42:24Z,308683,229686076
4,PG,$PG Market share grabs continue for Procter &a...,2020-07-21T19:31:05Z,2762379,229680970


In [ ]:
percentage_change = []
import itertools 
stock_dates = []
for (a, b, c) in zip(PG_Open, PG_Close, PG_Date): 
  change = ((b-a)/a)*100
  percentage_change.append((c, change))
  stock_dates.append(c)
# percentage_change


In [ ]:
print(percentage_change[:10])

[('2009-08-10', 0.4815100339564728), ('2009-08-11', -0.34542505573930865), ('2009-08-12', 0.15385000000000065), ('2009-08-13', 0.01912029032088937), ('2009-08-14', -0.34253092944873265), ('2009-08-17', -0.21024655963303518), ('2009-08-18', 0.09541793528939117), ('2009-08-19', 1.0139640329060582), ('2009-08-20', 0.17003589646703837), ('2009-08-21', 0.24321982706791723)]


In [ ]:
from datetime import datetime, timedelta, date
tweet_dates = []
date_PG = []
for date, mes in zip(PG_st_Date, PG_message):
  # date
  # d1 = .isoformat() + "Z"
  d1 = datetime.strptime(date, "%Y-%m-%dT%H:%M:%SZ")   #2020-07-21T21:37:30Z	
  # new_format = "%Y-%m-%d"
  # d1.isoformat()
  d2 = datetime.strftime(d1, "%Y-%m-%d")
  tweet_dates.append(d2)
  # print(type(d2))
  date_PG.append((d2, mes))

In [ ]:
print(len(percentage_change))

2759


In [ ]:
for da in tweet_dates:
  if da not in stock_dates:
    flag1 = False
    temp1_da = da
    while flag1 == False:
      temp1_da = datetime.strptime(temp1_da, "%Y-%m-%d") + timedelta(days=-1)
      temp1_da = datetime.strftime(temp1_da, "%Y-%m-%d")
      if temp1_da not in stock_dates:
        continue
      else:
        flag1=True
        flag2 = False
        temp2_da = da
        while flag2 == False:
          temp2_da = datetime.strptime(temp2_da, "%Y-%m-%d") + timedelta(days=1)
          # print('check3', temp2_da)
          temp2_da = datetime.strftime(temp2_da, "%Y-%m-%d")
          # print('check4', temp2_da)
          if temp2_da not in stock_dates:
            # print('check3')
            continue
          else:
            flag2= True
            day_before_change = 0
            day_after_change = 0
            count = 0
            for dat, per_change in percentage_change:
              count
              if temp1_da == dat:
                day_before_change = per_change
                count = count +1
              elif temp2_da == dat:
                day_after_change = per_change
                count = count +1
              if count == 2:
                
                mean_change = (day_before_change+day_after_change)/2
                # print(da,mean_change)
                stock_dates.append(da)
                percentage_change.append((da,mean_change))
                break
              else:
                continue
print(len(percentage_change))



3552


In [ ]:
for i in date_PG:
  print(i[0])

In [ ]:
PG_polarity = []
counter = 0
for date, pc in percentage_change:
  for datetweet, message in date_PG:

    if date == datetweet:
      if pc > 0.5:
        PG_polarity.append((message,1))
      elif pc < -0.5:
        PG_polarity.append((message, -1))
      else:
        PG_polarity.append((message, 0))

print(len(PG_polarity))
print(len(date_PG))


31494
31494


In [ ]:
message = []
label = []
for m,l in PG_polarity:
  message.append(m)
  label.append(l)
tweets = {'message':message,
          'label': label}
df = pd.DataFrame(tweets, columns = ['message', 'label'])
df.head()

,message,label
0,"Keep an eye on safety stocks here: $PG, $JNJ, ...",0
1,RT @ivolatility $PG calls active on 61K contra...,1
2,Pretty solid rotation into safety stocks today...,1
3,"$PG, massive call option action in jan 2010",0
4,"@weeklyTA $FSLR, $GLD, $PG please",1


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
df.to_csv('/content/gdrive/My Drive/Capstone/PG_label.csv', header=True, index=False, encoding='utf_8')

In [ ]:
def get_label(ch):
  if ch>0.5:
    return 1
  elif ch<-0.5:
    return -1
  else:
    return 0

In [ ]:
date_label = []
dates_index = []
for i in range(1,len(PG_Close)):
  label=99
  daydiff = datetime.strptime(PG_Date[i], "%Y-%m-%d") - datetime.strptime(PG_Date[i-1],"%Y-%m-%d")
  if daydiff.days == 1:
    change = ((PG_Close[i]-PG_Close[i-1])/PG_Close[i-1])*100
    if change>0.5:
      label = 1
    elif change<-0.5:
      label = -1
    else:
      label = 0
    date_label.append((PG_Date[i], change,label))
    dates_index.append(PG_Date[i])
  else:
    daysgap = daydiff.days
    daysadd = 1
    change_l = ((PG_Close[i]-PG_Close[i-1])/a)*100
    while daysgap >1:
      gapDate = datetime.strptime(PG_Date[i-1], "%Y-%m-%d") + timedelta(days=daysadd)
      change_g_mean = (change_l + date_label[-1][1])/2
      if change_g_mean>0.5:
        label = 1
      elif change_g_mean<-0.5:
        label = -1
      else:
        label = 0
      date_label.append((datetime.strftime(gapDate, "%Y-%m-%d"), change_g_mean,label))
      dates_index.append(datetime.strftime(gapDate, "%Y-%m-%d"))
      daysgap = daysgap - 1
      daysadd = daysadd + 1
    label = get_label(change_l)
    date_label.append((PG_Date[i], change_l,label))
    dates_index.append(PG_Date[i])

  # temp2_da = datetime.strftime(temp2_da, "%Y-%m-%d")

In [ ]:
print(dates_index[-100:])

['2020-04-16', '2020-04-17', '2020-04-18', '2020-04-19', '2020-04-20', '2020-04-21', '2020-04-22', '2020-04-23', '2020-04-24', '2020-04-25', '2020-04-26', '2020-04-27', '2020-04-28', '2020-04-29', '2020-04-30', '2020-05-01', '2020-05-02', '2020-05-03', '2020-05-04', '2020-05-05', '2020-05-06', '2020-05-07', '2020-05-08', '2020-05-09', '2020-05-10', '2020-05-11', '2020-05-12', '2020-05-13', '2020-05-14', '2020-05-15', '2020-05-16', '2020-05-17', '2020-05-18', '2020-05-19', '2020-05-20', '2020-05-21', '2020-05-22', '2020-05-23', '2020-05-24', '2020-05-25', '2020-05-26', '2020-05-27', '2020-05-28', '2020-05-29', '2020-05-30', '2020-05-31', '2020-06-01', '2020-06-02', '2020-06-03', '2020-06-04', '2020-06-05', '2020-06-06', '2020-06-07', '2020-06-08', '2020-06-09', '2020-06-10', '2020-06-11', '2020-06-12', '2020-06-13', '2020-06-14', '2020-06-15', '2020-06-16', '2020-06-17', '2020-06-18', '2020-06-19', '2020-06-20', '2020-06-21', '2020-06-22', '2020-06-23', '2020-06-24', '2020-06-25', '2020

In [ ]:
polarity = []
change = []
for index, row in df_stocktwitsPG.iterrows():
  
  d1 = datetime.strptime(row['datetime'], "%Y-%m-%dT%H:%M:%SZ")   #2020-07-21T21:37:30Z	
  d2 = datetime.strftime(d1, "%Y-%m-%d")
  d_index = dates_index.index(d2)
  polarity.append(date_label[d_index][2])
  change.append(date_label[d_index][1])
df_stocktwitsPG['change'] = change
df_stocktwitsPG['label'] = polarity

In [ ]:
df_stocktwitsPG[:40]

,symbol,message,datetime,user,message_id,change,label
0,PG,TOP DOW 30 Stocks with the highest Payout Rati...,2020-07-21T21:37:30Z,1550174,229729267,-0.133867,0
1,PG,$PG Green to red move: 1.11 percentage to -...,2020-07-21T20:28:32Z,1442893,229707089,-0.133867,0
2,PG,"$PG Bought. Hopefully, Ol&#39; Gil made the r...",2020-07-21T19:49:53Z,2926317,229689416,-0.133867,0
3,PG,$PG 322 overnight open 321 then down 319,2020-07-21T19:42:24Z,308683,229686076,-0.133867,0
4,PG,$PG Market share grabs continue for Procter &a...,2020-07-21T19:31:05Z,2762379,229680970,-0.133867,0
5,PG,"$PG Tough times never last, but tough people do.",2020-07-21T18:58:40Z,3814965,229666344,-0.133867,0
6,PG,Midday: $MDLZ $PG $BLK are our stock sugg...,2020-07-21T18:58:33Z,2142801,229666297,-0.133867,0
7,PG,$GNUS Some boards are littered with the worst ...,2020-07-21T15:42:57Z,1542918,229584306,-0.133867,0
8,PG,Premier Reports Q2 Production Results and Merc...,2020-07-21T10:09:06Z,210967,229451054,-0.133867,0
9,PG,Peak profit for the last 6 expired option aler...,2020-07-21T06:00:19Z,1442893,229434147,-0.133867,0
